In [5]:
import sys
import numpy as np
import matplotlib.pyplot as plt
from tkinter import filedialog
import tkinter as ERS
import winsound
import librosa
import librosa.display
import pandas as pd
file = "outputF.csv"
import pyaudio
import code
import IPython.display as ipd
import scipy.io.wavfile as wav
import warnings; 
warnings.simplefilter('ignore')

Result="?"

class LoginPage():
    def __init__(self):
        self.root = ERS.Tk()

        label = ERS.Label(self.root, text="Emotion Recognition System", fg="red", font=("Arial Bold", 20), bg = "white", anchor=W, justify=LEFT)
        label.grid(row=0)



        loadButton = ERS.Button(self.root, text="Load Audio", command=self.loadAudio, bg="white", fg="red")
        loadButton.grid(row=1, column=1, pady = 10, padx = 100)
        
        recordButton = ERS.Button(self.root, text="Record Audio", command=self.recordAudio, bg="white", fg="red")
        loadButton.grid(row=2, column=1, pady = 10, padx = 100)


        playButton = ERS.Button(self.root, text="Play Audio", command = self.playAudio, bg="white", fg="red")
        playButton.grid(row=3 , column=1, pady = 10, padx =100)

        procButton = ERS.Button(self.root, text="Extract Features & Classify", command=self.processAudio, bg="white", fg="red")
        procButton.grid(row=4, column=1, pady = 10, padx = 100)


        showSpectrogramButton = ERS.Button(self.root, text="Show Graph", command=self.showSpectrogram, bg="white", fg="red")
        showSpectrogramButton.grid(row=5, column=1, pady = 10, padx = 100)


        exitButton = ERS.Button(self.root, text="Quit", command=self.quitProgram, bg="white", fg="red")
        exitButton.grid(row=6, column=2, pady = 10, padx = 100)



        self.root.mainloop()

    def loadAudio(self):
        print("\t\t\t\t --Selecting File ..")
        global file_path
        file_path = filedialog.askopenfilename()
        print("\t --Loading Audio: " + file_path)
        return file_path

        # Need to call file_path somewhere so that it becomes global and accessible by every method

    def playAudio(self):
        print("\t\t\t\t --Playing File ..")
        winsound.PlaySound(file_path, winsound.SND_ASYNC)

        
    def recordAudio(self):
        RATE = 16000
        RECORD_SECONDS = 2.5
        CHUNKSIZE = 1024

        # initialize portaudio
        p = pyaudio.PyAudio()
        stream = p.open(format=pyaudio.paInt16, channels=1, rate=RATE, input=True, frames_per_buffer=CHUNKSIZE)
        print("***Recording ***")

        frames = []
        for _ in range(0, int(RATE / CHUNKSIZE * RECORD_SECONDS)):
            data = stream.read(CHUNKSIZE)
            frames.append(np.fromstring(data, dtype=np.int16))

        # Convert the list of numpy-arrays into a 1D array (column-wise)
        numpydata = np.hstack(frames)
        print("* done")
        # close stream
        stream.stop_stream()
        stream.close()
        p.terminate()

        ipd.Audio(numpydata, rate=RATE)

        dir = "testingData\\"
        filename = "output.wav"
        wav.write(dir + filename, RATE, numpydata)

    def processAudio(self):

        id = 1  # Song ID
        feature_set = pd.DataFrame()  # Feature Matrix

        # Individual Feature Vectors
        tempo_vector = pd.Series()
        average_beats = pd.Series()
        chroma_stft_mean = pd.Series()
        chroma_cq_mean = pd.Series()
        chroma_cens_mean = pd.Series()
        mel_mean = pd.Series()
        mfcc_mean = pd.Series()
        mfcc_delta_mean = pd.Series()
        rmse_value = pd.Series()
        energy_value = pd.Series()
        pow_value = pd.Series()


            # Reading Song

        y, sr = librosa.load(file_path, duration=5)
        S = np.abs(librosa.stft(y))

        # Extracting Features
        tempo, beats = librosa.beat.beat_track(y=y, sr=sr)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        chroma_cq = librosa.feature.chroma_cqt(y=y, sr=sr)
        chroma_cens = librosa.feature.chroma_cens(y=y, sr=sr)
        melspectrogram = librosa.feature.melspectrogram(y=y, sr=sr)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        mfcc_delta = librosa.feature.delta(mfcc)
        rmse = librosa.feature.rmse(y=y)
        energy = librosa.feature.melspectrogram(y=y, sr=sr, power=1)
        power = librosa.feature.melspectrogram(y=y, sr=sr, power=2)
        energy = librosa.core.amplitude_to_db(energy)
        power = librosa.core.power_to_db(power)

        # Transforming Features
        tempo_vector.at[id]=tempo
        average_beats.at[id]=np.average(beats)
        chroma_stft_mean.at[id]= np.mean(chroma_stft)  # chroma stft
        chroma_cq_mean.at[id] =np.mean(chroma_cq)  # chroma cq
        chroma_cens_mean.at[id]= np.mean(chroma_cens)  # chroma cens
        mel_mean.at[id]= np.mean(melspectrogram)  # melspectrogram
        mfcc_mean.at[id]= np.mean(mfcc)  # mfcc
        mfcc_delta_mean.at[id] =np.mean(mfcc_delta) # mfcc delta
        rmse_value.at[id]= np.mean(rmse)
        energy_value.at[id] =np.mean(energy)
        pow_value.at[id] =np.mean(power)
        print(file_path)

        # Concatenating Features into one csv and json format
        feature_set['tempo'] = tempo_vector  # tempo
        feature_set['average_beats'] = average_beats
        feature_set['chroma_stft_mean'] = chroma_stft_mean  # chroma stft
        feature_set['chroma_cq_mean'] = chroma_cq_mean  # chroma cq
        feature_set['chroma_cens_mean'] = chroma_cens_mean  # chroma cens
        feature_set['melspectrogram_mean'] = mel_mean  # melspectrogram
        feature_set['mfcc_mean'] = mfcc_mean  # mfcc
        feature_set['mfcc_delta_mean'] = mfcc_delta_mean  # mfcc delta
        feature_set['rmse_value'] = rmse_value  # rmse
        feature_set['energy_value'] = energy_value  # rmse
        feature_set['pow_value'] = pow_value  # rmse



        # Converting Dataframe into CSV Excel and JSON file
        feature_set.to_csv('result.csv')
        Result=code.code("result.csv")

    
    def showSpectrogram(self):

        y, sr = librosa.load(file_path)
        S = librosa.feature.melspectrogram(y, sr=sr, n_mels=128)
        log_S = librosa.amplitude_to_db(S, ref=np.max)
        plt.figure(figsize=(12, 4))
        librosa.display.specshow(log_S, sr=sr, x_axis='time', y_axis='mel')
        plt.title('mel power spectrogram')
        plt.colorbar(format='%+02.0f dB')
        plt.tight_layout()
        plt.show()

    def quitProgram(self):
        self.root.destroy()


LP = LoginPage()

NameError: name 'W' is not defined

In [ ]:
import django